In [1]:
from matplotlib import pyplot as plt
import numpy as np
plt.rcParams["figure.figsize"] = (20,10)

In [44]:
class Poly:
    """"
        c_0 + c_1 x + c_2 x^2 + .. + c_deg x^deg
    """
    def __init__(self, *, deg=2, coeffs=None):
        if coeffs is not None:
            self.coeffs = np.array(coeffs)
        else:
            self.coeffs = np.ones(deg+1)

    # def __isub__(self, ):

    def __call__(self, x):
        return sum(c_i * x**i for (i, c_i) in enumerate(self.coeffs))

    def grad(self, x):
        return np.array([x**i for (i, c_ip1) in enumerate(self.coeffs[:])]).T[0] # (i+1) * c_ip1 *

    def hessian(self, x):
        return np.diag(list((i+1) * c_ip1 * x**i for (i, c_ip1) in enumerate(self.coeffs[1:])))

In [80]:
def regression(x, y, method, **config):
    if config == {}:
        config = {"lr0": 0.5, "d": 0.005, "epoch": 1000}
    x_mat = np.hstack((np.ones((x.shape[0], 1)), x))
    k = x_mat.shape[1]
    f = lambda x_poly: y - x_poly(x)
    test = Poly(coeffs=np.full(len(x), 1))
    print(f'{np.array([test(x_i) for x_i in x])=}\n{y=}\n{f(test)=}')
    jacobi = lambda x_poly: np.array([- x_poly.grad(x[i]) for i in range(len(x))])
    bs = method(f, np.zeros(len(x)), **config, jacobi=jacobi)
    print('hm')
    print(f'came close by {f(Poly(coeffs=bs[-1]))}')
    # ax = plt.figure().add_subplot()
    # X = np.arange(len(bs))
    # ax.plot(X, np.vectorize(f(Poly(coeffs=bs[-1])))(*bs.T))
    # ax.grid()
    # if len(x[0]) == 1:
    #     draw_2d(x, y, bs[-1   ])
    return bs[-1]

def gauss_newton(f, x, *, lr, epoch, jacobi):
    points = np.zeros((epoch, len(x)))
    x_poly = Poly(coeffs=x)
    points[0] = x_poly.coeffs

    for i in range(1, epoch):
        j = jacobi(x_poly)
        print(f'{j=}\n{x_poly.coeffs=}\n{f(x_poly)=}\n{f(x_poly).reshape(-1,1)=}')
        # g = 2 * j * f(x_poly)
        g = np.matmul(j.T, f(x_poly).reshape(-1,1))
        h = np.matmul(j.T,j)
        p = (np.linalg.inv(h) * g)
        print(f'{g=}\n{h=}\n{p=}\n{np.linalg.inv(h)=}\n--------------------------------------------------------------------')
        # print(f'{p=}')
        # print(f'{x_poly.coeffs=}\n-\n{p=}')
        x_poly.coeffs -= lr * p
        points[i] = x_poly.coeffs
    return points

regression(np.array([[10], [-2.5], [10]]), np.array([0, 50, -100]), gauss_newton, lr=0.9, epoch=5)

np.array([test(x_i) for x_i in x])=array([[111.  ],
       [  4.75],
       [111.  ]])
y=array([   0,   50, -100])
f(test)=array([[-111.  ,  -61.  , -211.  ],
       [  -4.75,   45.25, -104.75],
       [-111.  ,  -61.  , -211.  ]])
j=array([[  -1.  ,  -10.  , -100.  ],
       [  -1.  ,    2.5 ,   -6.25],
       [  -1.  ,  -10.  , -100.  ]])
x_poly.coeffs=array([0., 0., 0.])
f(x_poly)=array([[   0.,   50., -100.],
       [   0.,   50., -100.],
       [   0.,   50., -100.]])
f(x_poly).reshape(-1,1)=array([[   0.],
       [  50.],
       [-100.],
       [   0.],
       [  50.],
       [-100.],
       [   0.],
       [  50.],
       [-100.]])


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 9 is different from 3)